In [1]:
import math
import pandas as pd
import numpy as np
from mpi4py import MPI
import os


# from kulfan import Kulfan
from ada.geometry.airfoils.kulfan import Kulfan

st = pd.ExcelFile('IEA-22-280-RWT_tabular.xlsx')

# ========================================================
# Load blade geometry parameters
# ========================================================
blade_geometry = pd.read_excel(st, 'Blade Geometry')

airfoils = blade_geometry.iloc[0:16]
airfoil_spans = airfoils['Spanwise position [r/R]'].to_numpy()
airfoil_names = airfoils['Airfoil name'].to_list()

detailed_geometry = blade_geometry.iloc[19:]
normSpans = detailed_geometry['Spanwise position [r/R]'].to_numpy()
chords    = detailed_geometry['Airfoil name'].to_numpy()
twists    = detailed_geometry['Unnamed: 2'].to_numpy()*180.0/np.pi
pas       = detailed_geometry['Unnamed: 3'].to_numpy()
spans     = detailed_geometry['Unnamed: 4'].to_numpy()
prebends  = detailed_geometry['Unnamed: 5'].to_numpy()
sweeps    = detailed_geometry['Unnamed: 6'].to_numpy()

# ========================================================
# Load airfoils
# ========================================================
airfoil_data = {}
for st_nm in st.sheet_names:
    if 'Airfoil ' in st_nm:
        coordinates = []
        nm = st_nm.replace('Airfoil ','')
        # print(nm)
        airfoil_sheet = pd.read_excel(st, st_nm)
        coordinates_end = 0
        for i in range(4,1000):
            coordinates_end = i
            rw = airfoil_sheet.iloc[i].to_numpy()
            if math.isnan(rw[0]):
                break
            else:
                coordinates.append([rw[0], rw[1]])
                
        coordinates = np.array(coordinates)
        coordinates[:,0] -= min(coordinates[:,0])
        airfoil_data[nm] = {}
        airfoil_data[nm]['coordinates'] = coordinates
        # plt.plot(coordinates[:,0], coordinates[:,1])
        # plt.axis('equal')
        
        airfoil_data[nm]['data'] = []
        currentDataStack = []
        for i in range(coordinates_end, len(airfoil_sheet)):
            rw = airfoil_sheet.iloc[i].to_numpy().tolist()
            if i == len(airfoil_sheet)-1:
                currentDataStack.append(rw)
                rw = ['alpha',0,0]
            
            if isinstance(rw[0], (float, int)):
                if not math.isnan(rw[0]):
                    currentDataStack.append(rw)
            elif isinstance(rw[0],str):
                if 'alpha' in rw[0]:
                    if len(currentDataStack)!=0:
                        dta = np.array(currentDataStack)
                        packedData = {'alpha':dta[:,0]*180.0/np.pi, 'cl':dta[:,1], 'cd':dta[:,2], 'cm':dta[:,3]}
                        airfoil_data[nm]['data'].append(packedData)
                        currentDataStack = []
                else:
                    # do nothing
                    assert('alpha' not in rw[0])
                    assert(isinstance(rw[0],str))

            else:
                assert(math.isnan(rw[0]) or rw[0]=='Reynolds')
        

[Codys-Air:35504] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.Codys-Air.501/jf.0/3716349952/sm_segment.Codys-Air.501.dd830000.0 could be created.


In [2]:
# ========================================================
# Make the Kulfan objects
# ========================================================
for nm, afl in airfoil_data.items():
    kfl = Kulfan()
    kfl.fit2coordinates(afl['coordinates'][:,0], afl['coordinates'][:,1], fit_order=10)
    airfoil_data[nm]['kulfan'] = kfl

fls = os.listdir('./oso1')
fls = list(sorted([f for f in fls if '.dat' in f]))
for f in fls:
    kfl = Kulfan()
    kfl.fit2file('./oso1/'+f)
    airfoil_data['oso-1-%s'%(f[-6:-4])] = {}
    airfoil_data['oso-1-%s'%(f[-6:-4])]['kulfan'] = kfl

airfoil_names[9]  = 'oso-1-36' 
airfoil_names[10] = 'oso-1-33' 
airfoil_names[11] = 'oso-1-30' 
airfoil_names[12] = 'oso-1-27' 
airfoil_names[13] = 'oso-1-24' 
airfoil_names[14] = 'oso-1-21' 
airfoil_names[15] = 'oso-1-21' 
print(airfoil_names)
print(airfoil_spans)

['circular', 'circular', 'FB90', 'FB80', 'FB70', 'FB60', 'SNL-FFA-W3-560FB', 'SNL-FFA-W3-480FB', 'SNL-FFA-W3-420FB', 'oso-1-36', 'oso-1-33', 'oso-1-30', 'oso-1-27', 'oso-1-24', 'oso-1-21', 'oso-1-21']
[0 0.02 0.04868313468735217 0.06649308419703373 0.08345591801468194
 0.1022090419357058 0.1103627201416439 0.1364246061019374
 0.1556444058751519 0.196653365754448 0.2469614873536471
 0.3992636115637571 0.5335887750152993 0.738938689884722
 0.9799991709122947 1]


In [3]:
# ========================================================
# Stack all the detailed airfoils
# ========================================================
esp_string = 'mark \n'

circle_string = ''
theta = 25 * np.pi/180.0
circle_string += 'skbeg %f %f 0 \n'%(np.cos(theta), np.sin(theta))
circle_string += 'cirarc 0  1 0 -1 0 0 \n'
circle_string += 'cirarc 0 -1 0 %f %f 0 \n'%(np.cos(theta), -1*np.sin(theta))
circle_string += 'cirarc 1 0 0 %f %f 0 \n'%(np.cos(theta), np.sin(theta))
circle_string += 'skend \n'
circle_string += 'scale 0.5 \n'
circle_string += 'translate 0.5 0 0 \n'

# FB80 has a larger TE gap than FB90, which leads to significant blending issues
# this is in a region where flow is slow
airfoil_data['FB90']['kulfan'].constants.TE_gap *= 1.3

for i in range(0,len(normSpans)):
    normSpan = normSpans[i]
    chord    = chords[i]
    twist    = twists[i]
    pa       = pas[i]
    span     = spans[i]
    prebend  = prebends[i]
    sweep    = sweeps[i]
    
    if normSpan <= 0.02:
        esp_string += circle_string

    elif normSpan <= airfoil_spans[2]:
        esp_string += airfoil_data['FB90']['kulfan'].toESP()
        
    else:

        if normSpan != 1:
            right_airfoil_index = np.argmax(airfoil_spans>normSpan)
            left_airfoil_index  = right_airfoil_index - 1
        else:
            right_airfoil_index = -1
            left_airfoil_index  = -1
            
        interval_length = airfoil_spans[right_airfoil_index] - airfoil_spans[left_airfoil_index]
        if interval_length == 0:
            progres_frac = 1
        else:    
            progress_frac = (normSpan - airfoil_spans[left_airfoil_index]) / interval_length
        
        xcoords_left  = airfoil_data[airfoil_names[left_airfoil_index]]['kulfan'].xcoordinates
        xcoords_right = airfoil_data[airfoil_names[right_airfoil_index]]['kulfan'].xcoordinates
        ycoords_left  = airfoil_data[airfoil_names[left_airfoil_index]]['kulfan'].ycoordinates
        ycoords_right = airfoil_data[airfoil_names[right_airfoil_index]]['kulfan'].ycoordinates
        xcoords = (1-progress_frac)*xcoords_left + progress_frac*xcoords_right
        ycoords = (1-progress_frac)*ycoords_left + progress_frac*ycoords_right
        
        kfl = Kulfan()
        kfl.fit2coordinates(xcoords, ycoords)
        esp_string += kfl.toESP()
        
    esp_string += 'translate %f 0 0 \n'%(-1*pa)
    esp_string += 'scale %f \n'%(chord)
    esp_string += 'rotatez %f 0 0 \n'%(twist)
    esp_string += 'translate 0 %f %f \n'%(-prebend,span)
    esp_string += '\n'


esp_string += "dimension custom 1 3 0\n"
esp_string += 'set custom "0.040;0.650;0.970"\n'
# esp_string += "rule 1 \n"
esp_string += "blend 1 \n"
# esp_string += "attribute _color custom\n"
esp_string += 'store blade \n'
esp_string += 'restore blade \n'
esp_string += 'box -10 -10 -10 20 20 %f \n'%(0.196653366 * 137.8)
esp_string += 'intersect \n'
esp_string += "attribute _color $yellow\n"
esp_string += 'store root \n'
# esp_string += 'restore blade \n'
# esp_string += 'box -10 -10 -10 20 20 %f \n'%(0.196653366 * 137.8)
# esp_string += 'subtract \n'
# esp_string += 'restore root\n'

# ========================================================
# Stack all the detailed airfoils
# ========================================================
esp_string += 'mark \n'

circle_string = ''
theta = 25 * np.pi/180.0
circle_string += 'skbeg %f %f 0 \n'%(np.cos(theta), np.sin(theta))
circle_string += 'cirarc 0  1 0 -1 0 0 \n'
circle_string += 'cirarc 0 -1 0 %f %f 0 \n'%(np.cos(theta), -1*np.sin(theta))
circle_string += 'cirarc 1 0 0 %f %f 0 \n'%(np.cos(theta), np.sin(theta))
circle_string += 'skend \n'
circle_string += 'scale 0.5 \n'
circle_string += 'translate 0.5 0 0 \n'

# FB80 has a larger TE gap than FB90, which leads to significant blending issues
# this is in a region where flow is slow
# airfoil_data['FB90']['kulfan'].constants.TE_gap *= 1.3

for i in range(0,len(normSpans)):
    normSpan = normSpans[i]
    chord    = chords[i]
    twist    = twists[i]
    pa       = pas[i]
    span     = spans[i]
    prebend  = prebends[i]
    sweep    = sweeps[i]
    
    if normSpan <= 0.02:
        esp_string += circle_string

    elif normSpan <= airfoil_spans[2]:
        esp_string += airfoil_data['FB90']['kulfan'].toESP()
        
    else:

        if normSpan != 1:
            right_airfoil_index = np.argmax(airfoil_spans>normSpan)
            left_airfoil_index  = right_airfoil_index - 1
        else:
            right_airfoil_index = -1
            left_airfoil_index  = -1
            
        interval_length = airfoil_spans[right_airfoil_index] - airfoil_spans[left_airfoil_index]
        if interval_length == 0:
            progres_frac = 1
        else:    
            progress_frac = (normSpan - airfoil_spans[left_airfoil_index]) / interval_length
        
        xcoords_left  = airfoil_data[airfoil_names[left_airfoil_index]]['kulfan'].xcoordinates
        xcoords_right = airfoil_data[airfoil_names[right_airfoil_index]]['kulfan'].xcoordinates
        ycoords_left  = airfoil_data[airfoil_names[left_airfoil_index]]['kulfan'].ycoordinates
        ycoords_right = airfoil_data[airfoil_names[right_airfoil_index]]['kulfan'].ycoordinates
        xcoords = (1-progress_frac)*xcoords_left + progress_frac*xcoords_right
        ycoords = (1-progress_frac)*ycoords_left + progress_frac*ycoords_right
        
        kfl = Kulfan()
        kfl.fit2coordinates(xcoords, ycoords)
        esp_string += kfl.toESP()
        
    esp_string += 'translate %f 0 0 \n'%(-1*pa)
    esp_string += 'scale %f \n'%(chord)
    esp_string += 'rotatez %f 0 0 \n'%(twist)
    esp_string += 'translate 0 %f %f \n'%(-prebend,span)
    esp_string += '\n'


esp_string += "dimension custom 1 3 0\n"
esp_string += 'set custom "0.040;0.650;0.970"\n'
# esp_string += "rule 1 \n"
esp_string += "blend 1 \n"
esp_string += "attribute _color custom\n"
esp_string += 'store blade \n'
# esp_string += 'restore blade \n'
# esp_string += 'box -10 -10 -10 20 20 %f \n'%(0.196653366 * 137.8)
# esp_string += 'intersect \n'
# esp_string += "attribute _color $yellow\n"
# esp_string += 'store root \n'
esp_string += 'restore blade \n'
esp_string += 'box -10 -10 -10 20 20 %f \n'%(0.196653366 * 137.8)
esp_string += 'subtract \n'
esp_string += 'restore root\n'





# # esp_string += 'blend 1 \n'
# esp_string += 'rule 1 \n'
# # esp_string += 'dimension tess_params 1 3 0 \n'
# # esp_string += 'set tess_params "1.0; 0.1; 10;" \n'
# # esp_string += 'store blade \n'
# # esp_string += 'restore blade \n'
# # esp_string += 'attribute .tParams tess_params \n'
# # esp_string += 'attribute .color blue \n'




f = open('22mw_test_fullDetail_oso.csm','w')
f.write(esp_string)
f.close()